## Gate 鉴权机制与 Rust sha512 踩坑记

gate websocket v4 鉴权比较简单，用文档中的例子也能跑通

但是 gate restful v4 的鉴权代码我翻译成 Rust 之后下单接口报错 401 Signature mismatch

用同样的参数输入到 python/rust 的 gen_sign 方法中生成的签名一样，于是我用 postman-echo 测试下二者请求的差异

以下是成功的 python 下单请求，返回 `400 {"label":"INVALID_PARAM_VALUE","message":"Your order size 0.000100 USDT is too small. The minimum is 1 USDT"}`

```json
{
  "data": {
    "amount": "0.0001",
    "currency_pair": "BTC_USDT",
    "price": "1",
    "side": "buy"
  },
  "form": {},
  "headers": {
    "host": "postman-echo.com",
    "user-agent": "python-requests/2.22.0",
    "key": "...",
    "timestamp": "1697024389",
    "sign": "9af63b7a8c08909b08d7e291cd14c7804000f882b1859524934214f26acb1fe5083a45f2e0710dfe7767cb8c2ef6c9a407edf8b975b04afea7a77948df4961d6",
    "content-type": "application/json"
  },
  "json": {
    "amount": "0.0001",
    "currency_pair": "BTC_USDT",
    "price": "1",
    "side": "buy"
  }
}
```


In [25]:
import configparser
config = configparser.ConfigParser()
config.read("config.toml")
api_key = config["gate"]["api_key"][1:-1]
api_secret = config["gate"]["api_secret"][1:-1]

In [26]:
import time
import hashlib
import hmac
def gen_sign(method, url, query_string=None, payload_string=None):
    time_ = int(time.time())

    m = hashlib.sha512()
    m.update((payload_string or "").encode())
    hashed_payload = m.hexdigest()
    s = f'{method}\n{url}\n{query_string or ""}\n{hashed_payload}\n{time_}'
    sign = hmac.new(api_secret.encode(), s.encode(), hashlib.sha512).hexdigest()

    print("-"*20)
    print(payload_string)
    print(s)
    print(sign)
    print("-"*20)

    return {'KEY': api_key, 'Timestamp': str(time_), 'SIGN': sign}

In [30]:
import requests
if True:
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    url = '/spot/orders'
else:
    host = "https://postman-echo.com"
    prefix = ''
    url = '/post'
common_headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
request_content = '{"amount":"0.0001","currency_pair":"BTC_USDT","price":"1","side":"buy"}'
sign_headers = gen_sign('POST', prefix + url, "", request_content)
sign_headers.update(common_headers)
res = requests.post(host + prefix + url, headers=sign_headers, data=request_content)
print(res.text)

--------------------
{"amount":"0.0001","currency_pair":"BTC_USDT","price":"1","side":"buy"}
POST
/api/v4/spot/orders

c74dec43a6742ba02e554c0d2ac889fe6391014fc5eb226798e741f1422d3b7dfe2e2a1df31b158cff3d32a8fe0fdddb6fa74b38fa893234c8ac0d00f1a4edb8
1697027893
70921d20521bb685f4b9a696552ef17258fe33f3eaff640e146219fe6f41cea5d189a2666bfb8feebea9730702b268618fabdcfb36441d6c80557d61ed721229
--------------------
{"label":"INVALID_PARAM_VALUE","message":"Your order size 0.000100 USDT is too small. The minimum is 1 USDT"}


In [31]:
!time=1697027893 cargo test --package trade --lib -- market::gate::test_gate_post_order --nocapture --ignored

    Finished test [unoptimized + debuginfo] target(s) in 0.06s
     Running unittests src/lib.rs (target/debug/deps/trade-b987235658e21c77)

running 1 test
{"amount":"0.0001","currency_pair":"BTC_USDT","price":"1","side":"buy"}
POST
/api/v4/spot/orders

c74dec43a6742ba02e554c0d2ac889fe6391014fc5eb226798e741f1422d3b7dfe2e2a1df31b158cff3d32a8fe0fdddb6fa74b38fa893234c8ac0d00f1a4edb8
1697027893
260be2a4159221c3c6b970d5383c7016f3f0aba174036c247b6bcf093046ad75af718f21715d490400399cbf4207473267858a4fae0512bfaf499a2ef475cbd6
[src/market/gate.rs:417] status = 401
{"label":"INVALID_SIGNATURE","message":"Signature mismatch"}

thread 'market::gate::test_gate_post_order' panicked at src/market/gate.rs:420:9:
assertion failed: status.is_success()
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
test market::gate::test_gate_post_order ... FAILED

failures:

failures:
    market::gate::test_gate_post_order

test result: FAILED. 0 passed; 1 failed; 0 ignored; 0 measured; 15

很奇怪的是，Rust 的 hexdigiest 算错了，如果拿同样的入参给 Rust 去算，又能得出跟 python3 一样正确的结果。

- <https://stackoverflow.com/questions/72872350/hmac-sha512-in-rust-cant-get-expected-result>

因为签名函数内进行两次 hash 我怀疑 rust 的 sha512 库是不是用了一些静态变量之类的有状态的导致同一个函数内 hash/digest 两次使得第二次结果结果无法算对

trade-demo 项目 13f84dbb 版本 src/market/gate.rs `fn restful_apiv4_gen_sign`

```rust
let mut hasher = Sha512::new();
hasher.update(body.as_bytes());
let digest = hasher.finalize();
let hashed_payload = format!("{digest:02x}");

let s = format!("{method}\n{url}\n{query_string}\n{hashed_payload}\n{time}");
let mut mac = Hmac::<Sha512>::new_from_slice(CONFIG.gate.api_secret.as_bytes()).unwrap();
mac.update(s.as_bytes());
let result = mac.finalize().into_bytes();
let sign = format!("{result:02x}");
```

于是我尝试将函数内两次 digest **隔离** 成两个函数，结果签名校验直接通过了

[sha2 源码](https://docs.rs/crate/sha2/latest/source/src/sha512/x86.rs) 基本上是一半全是汇编指令，Rust-Crypto 组织做的 sha2 这样加密库比较激进用汇编提升性能，在 Rust 社区颇有人气库的下载量很高

所以很可能就是运算过程中没有清理干净上次哈希运算残留的 avx 向量寄存器状态，在 tokio 线程频繁线程上下文切换时，导致同个函数内第二次进行哈希运算的结果就错误了

解决办法就这种用 AVX 指令集去哈希运算的每次运算都在一个独立的函数中，我猜测原因是一次哈希运算函数上下文状态保存/恢复都会清理向量寄存器的状态因此解决了问题

但是这个 Bug 需要 tokio 网络请求 复杂异步环境中才能复现，最后我问 gpt

> x86的向量寄存器，在线程切换的时候会自动清理/保存之前的状态吗

```
x86架构的向量寄存器（如SSE和AVX寄存器）在线程切换时不会自动清理或保存之前的状态。这些向量寄存器的内容是与特定线程相关的，并且在线程切换时，寄存器的状态不会自动保存或恢复。

在线程切换时，操作系统负责保存和恢复通用寄存器（如EAX，EBX，ECX等），以确保线程能够正确恢复执行。然而，对于向量寄存器，由于其状态与特定线程有关，操作系统并不会自动为其保存和恢复状态。

因此，在多线程编程中，如果你使用了向量寄存器，你需要自己负责在线程切换前保存寄存器的状态，并在切换回来时恢复其状态
```

> x86 Linux 函数调用的时候，会自动保存/恢复向量寄存器的状态吗？

```
在x86架构的Linux系统中，通常情况下在函数调用时，向量寄存器的状态不会自动保存或恢复。和上述提到的线程切换类似，函数调用时只有通用寄存器的状态会被保存和恢复，而向量寄存器的状态需由开发者自行管理
```

我最后的思考是，关闭公司电脑台式机的超线程会不会降低向量寄存器状态的风险？gpt 的回答是可能会降低

---

最后发现竟然是自己小丑了，sha256 运算一定是无状态的，Gate 所的密钥代码里面传入的是币安的密钥，难怪单元测试里面传入一个写死的密钥能通过，正式代码不通过